## Goal
This notebook is set up to train a MaskRCNN model using Ray Tune for hyperparameter tuning

In [1]:
## Imports
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader
import numpy as np
from pathlib import Path
import os
import math
import sys
import tempfile
from functools import partial

# Model Transforms
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F

# Model imports
import torch
import torchvision
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection import FasterRCNN

# Data imports
from facet_ml.classification import mask_rcnn
from pathlib import Path
import os

# Ray tune imports
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle

import matplotlib.pyplot as plt

device = "cuda"

c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-12 12:11:10,304	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-12 12:11:10,483	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
### Model Creation Functions ###
def get_model_instance_segmentation(num_classes,
                                    config:dict):
    
    ## load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    ## Make an updated FastRCNN with backgone changes as needed
    backbone = config.get("backbone","resnet50")
    if backbone == "mobilenet_v2":
        backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
        backbone.out_channels = 1280
        fast_rcnn = FasterRCNN(backbone, in_features=in_features, num_classes=num_classes,)
        model.roi_heads.box_predictor = fast_rcnn
    else:
        # Use defualt resnet50 bacbone
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

def get_model_instance_segmentation(num_classes,config):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(
        in_features_mask,
        hidden_layer,
        num_classes
    )

    return model

def get_optimizer(model,config):
    '''
    Get an optimzier based on the config settings
    '''
    params = [p for p in model.parameters() if p.requires_grad]
    if config["optimizer"] == "Adam":
        optimizer = torch.optim.Adam(params,
                         lr=config["lr"],
                        weight_decay=config["weight_decay"],
                        betas=config["betas"]
            )
    elif config["optimizer"] == "SGD":
        optimizer = torch.optim.SGD(
            params,
            lr=config["lr"],
            momentum=config["momentum"],
            weight_decay=config["weight_decay"] # .0005 starting
        )
    
    return optimizer

def get_scheduler(optimizer,config):
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=3,
        gamma=config["gamma"]
    )
    return lr_scheduler

In [3]:
### Load Transforms and Data ###
def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomVerticalFlip(0.5))
        transforms.append(T.RandomRotation(90))
        transforms.append(T.RandomResizedCrop(size=256, scale=(0.6, 1.4)))
        T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.0)
        # transforms.append(T.RandomCrop(size=(224, 224)))


    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

def load_colloidal_data(data_dir=r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5"
):
    train_dir = Path(data_dir) / "train"
    test_dir  = Path(data_dir) / "test"
    cd_train = mask_rcnn.ManualCocoColloidalDataset(
        str(train_dir),
        str(train_dir / "_annotations.coco.json"),
        transforms=get_transform(True)
    )
    cd_test = mask_rcnn.ManualCocoColloidalDataset(
        str(test_dir),
        str(Path(test_dir) / "_annotations.coco.json"),
        transforms=get_transform(False)
    )


    return cd_train, cd_test

In [4]:
## Ray Tune config parameters
ray_config ={
    # Opt choice
    "optimizer": tune.choice(["Adam","SGD"]),

    # Region Choices

    # General choices
    "lr": tune.loguniform(1e-4,1e-1),
    "betas":  tune.choice([(0.9, 0.999), (0.5, 0.999)]),
    "momentum": tune.uniform(0.5, 0.9),
    "weight_decay": tune.loguniform(1e-4,1e-1),
    "gamma": tune.uniform(0.1, 0.9)
}

In [5]:
## Ray Tune Functions ##

def train_colloidal(config,data_dir):
    '''
    Ray tune train loop
    '''
    device = "cuda"

    model = get_model_instance_segmentation(2,config)
    model.to(device)
    optimizer = get_optimizer(model, config)
    scheduler = get_scheduler(optimizer, config)

    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "rb") as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state["epoch"]
            model.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    trainset, testset = load_colloidal_data(data_dir)
    def collate_fn(batch):
        '''
        Collation function receives [(image_1, targets_1{masks,boxes,labels}), (image_10, targets_1{masks,boxes,labels})...]
        Need to stack image_1
        '''
        images = [item[0] for item in batch]
        targets = [item[1] for item in batch]
        # return tuple(zip(*batch))
        return images,targets
    
    train_loader = DataLoader(
        trainset,
        batch_size=2,
        num_workers=0,
        collate_fn=collate_fn
    )
    test_loader = DataLoader(
        testset,
        batch_size=2,
        num_workers=0,
        collate_fn=collate_fn
    )
    for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            images, targets = data
            images = list(image.to(device) for image in images)
            targets = [
                {
                    k: v.to(device) if isinstance(v, torch.Tensor) else v
                    for k, v in t.items()
                }
                for t in targets
            ]
            with torch.cuda.amp.autocast(enabled=False):
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

            losses_reduced = sum(loss for loss in loss_dict.values())
            loss_value = losses_reduced.item()

            print(losses)
            if not math.isfinite(loss_value):
                print(f"Loss is {loss_value}, stopping training")
                # print(loss_dict_reduced)
                sys.exit(1)
            
            losses.backward()
            optimizer.step()

            if scheduler is not None:
                scheduler.step()

            test_loss = 0.0
            test_steps = 0
            total = 0
            correct = 0

            for i, data in enumerate(test_loader,0):
                with torch.no_grad():
                    images, targets = data
                    images = list(image.to(device) for image in images)
                    targets = [
                        {
                            k: v.to(device) if isinstance(v, torch.Tensor) else v
                            for k, v in t.items()
                        }
                        for t in targets
                    ]
                    loss_dict = model(images, targets)
                    losses_reduced = sum(loss for loss in loss_dict.values())
                    test_loss += losses_reduced
                    test_steps += 1
            
            checkpoint_data = {
                "epoch": epoch,
                "net_state_dict":model.state_dict(),
                "optimizer_state_dict":optimizer.state_dict()
            }
            with tempfile.TemporaryDirectory() as checkpoint_dir:
                data_path = Path(checkpoint_dir) / "data.pkl"
                with open(data_path, "wb") as fp:
                    pickle.dump(checkpoint_data, fp)

                checkpoint = Checkpoint.from_directory(checkpoint_dir)
                train.report(
                    {"loss": test_loss.to("cpu").detach().numpy() / test_steps, 
                    #  "accuracy": correct / total
                     },
                    checkpoint=checkpoint,
                )
    print("Finished Training")

In [14]:
data_dir = os.path.abspath(r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5")
print(ray_config["optimizer"])
test_config ={
    # Opt choice
    "optimizer": "SGD",

    # Region Choices

    # General choices
    "lr": 0.005,
    "betas":  (0.9, 0.999),
    "momentum":0.9,
    "weight_decay": .5,
    "gamma": 0.6
}
train_colloidal(test_config,data_dir=data_dir)

c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\ray\train\_internal\session.py:652: UserWarning: `get_checkpoint` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(
c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\tv_tensors\_tv_tensor.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  return torch.as_tensor(data, dtype=dtype, device=device).requires_grad_(requires_grad)


tensor(8.9019, device='cuda:0', grad_fn=<AddBackward0>)


c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\ray\train\_internal\session.py:652: UserWarning: `report` is meant to only be called inside a function that is executed by a Tuner or Trainer. Returning `None`.
  warnings.warn(


tensor(3.1666, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.5968, device='cuda:0', grad_fn=<AddBackward0>)
tensor(35.0724, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.3626, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9982.0693, device='cuda:0', grad_fn=<AddBackward0>)
tensor(nan, device='cuda:0', grad_fn=<AddBackward0>)
Loss is nan, stopping training


SystemExit: 1

c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
def main(config, num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath(r"C:\Users\Jacob\Desktop\Academics\Mirkin\colloidal_crystal_ML\ProcessedData\Coco_v5")
    load_colloidal_data(data_dir)
    # config = {
    #     "l1": tune.choice([2**i for i in range(9)]),
    #     "l2": tune.choice([2**i for i in range(9)]),
    #     "lr": tune.loguniform(1e-4, 1e-1),
    #     "batch_size": tune.choice([2, 4, 8, 16]),
    # }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )

    def short_dirname(trial):
        return "trial_" + str(trial.trial_id)
    
    result = tune.run(
        partial(train_colloidal, data_dir=data_dir),
        # resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        trial_dirname_creator=short_dirname,
        max_concurrent_trials=4
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    # print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

In [8]:
main(ray_config,num_samples=30)

2024-09-12 12:11:12,864	INFO worker.py:1783 -- Started a local Ray instance.
2024-09-12 12:11:14,054	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-09-12 12:11:14,055	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(func pid=10444) c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\tv_tensors\_tv_tensor.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
(func pid=10444)   return torch.as_tensor(data, dtype=dtype, device=device).requires_grad_(requires_grad)


(func pid=10444) tensor(8.0813, device='cuda:0', grad_fn=<AddBackward0>)
(func pid=20128) tensor(7.9802, device='cuda:0', grad_fn=<AddBackward0>)
(func pid=19260) tensor(8.3867, device='cuda:0', grad_fn=<AddBackward0>)
(func pid=34412) tensor(5.9127, device='cuda:0', grad_fn=<AddBackward0>)


Trial name,loss,should_checkpoint
train_colloidal_03f42_00000,inf,True
train_colloidal_03f42_00001,3.10163,True
train_colloidal_03f42_00002,3.02893,True
train_colloidal_03f42_00003,1.10258e+08,True
train_colloidal_03f42_00004,16.8359,True
train_colloidal_03f42_00005,3.30357,True
train_colloidal_03f42_00006,2.09828e+10,True
train_colloidal_03f42_00007,6.5979,True
train_colloidal_03f42_00008,nan,True
train_colloidal_03f42_00009,7.08466e+06,True


(func pid=10444) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14/trial_03f42_00002/checkpoint_000000)
(func pid=19260) c:\Users\Jacob\miniconda3\envs\colloidal_crystal_env\lib\site-packages\torchvision\tv_tensors\_tv_tensor.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.) [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(func pid=19260)   return torch.as_tensor(data, dtype=dtype, device=device).requires_grad_(requires_grad) [repeated 3x across cluster]
2024-09-12 12:15:06

...

(func pid=19364) tensor(3.2521, device='cuda:0', grad_fn=<AddBackward0>)
(func pid=28928) tensor(9.3012, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-12 14:10:31,026	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'betas': (0.9, 0.999)}
(func pid=19364) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14/trial_03f42_00025/checkpoint_000003)
2024-09-12 14:10:32,797	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'betas': (0.9, 0.999)}
(func pid=28928) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14/trial_03f42_00029/checkpoint_000000)


(func pid=10124) tensor(3.4085, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-12 14:10:41,919	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'betas': (0.5, 0.999)}
2024-09-12 14:10:41,941	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14' in 0.0205s.
(func pid=10124) Checkpoint successfully created at: Checkpoint(filesystem=local, path=C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14/trial_03f42_00028/checkpoint_000001)


TuneError: ('Trials did not complete', [train_colloidal_03f42_00008, train_colloidal_03f42_00015, train_colloidal_03f42_00022, train_colloidal_03f42_00023])

In [13]:
from ray.tune import ExperimentAnalysis
folder_path = r"C:/Users/Jacob/ray_results/train_colloidal_2024-09-12_12-11-14" 
analysis = ExperimentAnalysis(folder_path)

completed_trials = [trial for trial in analysis.trials if trial.status == "TERMINATED"]
failed_trials = [trial for trial in analysis.trials if trial.status == "ERROR"]

best_trial = min(completed_trials, key=lambda trial: trial.last_result["loss"])

# Print the best performing trial and its results
print("Best trial config: ", best_trial.config)
print("Best trial final result: ", best_trial.last_result)

Best trial config:  {'optimizer': 'Adam', 'lr': 0.00016517420107310982, 'betas': [0.5, 0.999], 'momentum': 0.5277638298935976, 'weight_decay': 0.0006188018644099798, 'gamma': 0.512368076225412}
Best trial final result:  {'loss': 2.4554710388183594, 'timestamp': 1726167629, 'checkpoint_dir_name': 'checkpoint_000009', 'should_checkpoint': True, 'done': True, 'training_iteration': 10, 'trial_id': '03f42_00012', 'date': '2024-09-12_14-00-29', 'time_this_iter_s': 38.13473606109619, 'time_total_s': 319.3866858482361, 'pid': 22496, 'hostname': 'DESKTOP-RD74FOL', 'node_ip': '127.0.0.1', 'config': {'optimizer': 'Adam', 'lr': 0.00016517420107310982, 'betas': [0.5, 0.999], 'momentum': 0.5277638298935976, 'weight_decay': 0.0006188018644099798, 'gamma': 0.512368076225412}, 'time_since_restore': 319.3866858482361, 'iterations_since_restore': 10, 'experiment_tag': '12_betas=0_5_0_999,gamma=0.5124,lr=0.0002,momentum=0.5278,optimizer=Adam,weight_decay=0.0006'}
